In [2]:
import pandas as pd 
import numpy as np 

# Find similar players across leagues using clustering, or FP Mining. 
- Compute the players stats into vectors by normalizing the values 
- We get a feature_vector for each player, project that to an embedding space and perform clutersing 

In [3]:
pl_players = pd.read_csv('./data/premier_league_data_fb_ref.csv')
bundesliga_players = pd.read_csv('./data/bundesliga_data_fb_ref.csv')
la_liga_players = pd.read_csv('./data/la_liga_data_fb_ref.csv')
ligue_1_players = pd.read_csv('./data/ligue_1_data_fb_ref.csv')
serie_a_players = pd.read_csv('./data/serie_a_data_fb_ref.csv')

pl_season_data = pl_players.groupby(['Season'])
bundesliga_season_data = bundesliga_players.groupby(['Season'])
la_liga_season_data = la_liga_players.groupby(['Season'])
ligue_1_season_data = ligue_1_players.groupby(['Season'])
serie_a_season_data = serie_a_players.groupby(['Season'])


In [4]:
pl_chelsea_players = pl_players[pl_players['Squad'] == 'Chelsea']
pl_chelsea_players.head()

,Unnamed: 0,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
33,178,Andreas Christensen,dk DEN,DF,Chelsea,eng Premier League,21,1996.0,27,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,232,Antonio RÃ¼diger,de GER,DF,Chelsea,eng Premier League,24,1993.0,27,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,384,Callum Hudson-Odoi,eng ENG,FW,Chelsea,eng Premier League,16,2000.0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,408,Cesc FÃ bregas,es ESP,MF,Chelsea,eng Premier League,30,1987.0,32,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,422,Charly Musonda Jr.,be BEL,FW,Chelsea,eng Premier League,20,1996.0,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Lets find the super sub

In [7]:
pl_agg_stats = pd.DataFrame()
pl_agg_stats['Player'] = pl_players['Player']
pl_agg_stats['Nation'] = pl_players['Nation']
pl_agg_stats['Pos'] = pl_players['Pos']
pl_agg_stats['Squad'] = pl_players['Squad']
pl_agg_stats['Comp'] = pl_players['Comp']
pl_agg_stats['Age'] = pl_players['Age']
pl_agg_stats['Season'] = pl_players['Season']
pl_agg_stats['OffTheBench'] = pl_players['MP'] - pl_players['Starts']

pl_agg_stats.head()


,Player,Nation,Pos,Squad,Comp,Age,Season,OffTheBench
0,Aaron Cresswell,eng ENG,DF,West Ham,eng Premier League,27,2017-2018,1
1,Aaron Lennon,eng ENG,MF,Burnley,eng Premier League,30,2017-2018,1
2,Aaron Lennon,eng ENG,"FW,MF",Everton,eng Premier League,30,2017-2018,6
3,Aaron Mooy,au AUS,MF,Huddersfield,eng Premier League,26,2017-2018,2
4,Aaron Ramsey,wls WAL,MF,Arsenal,eng Premier League,26,2017-2018,3


# Normalize Numeric Columns

In [5]:
pl_chelsea_numeric_col = pl_chelsea_players.select_dtypes(include='number')
# pl_chelsea_numeric_col.head()
pl_chelsea_norm = (pl_chelsea_numeric_col - pl_chelsea_numeric_col.mean()) / (pl_chelsea_numeric_col.std())
pl_chelsea_players[pl_chelsea_numeric_col.columns] = pl_chelsea_norm
pl_chelsea_players['Age'] = pl_players['Age']
pl_chelsea_players['Born'] = pl_players['Born']
pl_chelsea_players.fillna(0.0)
pl_chelsea_players.head()

/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_54185/3484893783.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_chelsea_players[pl_chelsea_numeric_col.columns] = pl_chelsea_norm
/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_54185/3484893783.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_chelsea_players['Age'] = pl_players['Age']
/var/folders/qw/9j8hm051443c2lydt69r_n7w0000gn/T/ipykernel_54185/3484893783.py:6: SettingWithCopyWarning: 
A value is trying to be set on a c

,Unnamed: 0,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
33,-1.683119,Andreas Christensen,dk DEN,DF,Chelsea,eng Premier League,21,1996.0,0.716603,0.717792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,-1.669651,Antonio RÃ¼diger,de GER,DF,Chelsea,eng Premier League,24,1993.0,0.716603,0.878929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,-1.631742,Callum Hudson-Odoi,eng ENG,FW,Chelsea,eng Premier League,16,2000.0,-1.179781,-1.135283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,-1.625756,Cesc FÃ bregas,es ESP,MF,Chelsea,eng Premier League,30,1987.0,1.095880,0.878929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,-1.622265,Charly Musonda Jr.,be BEL,FW,Chelsea,eng Premier League,20,1996.0,-1.103925,-1.135283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
flattened_list_cheslea = pl_chelsea_players[:].values.tolist()
flattened_list_cheslea[0]
fv_chelsea = list()
for l in flattened_list_cheslea:
    k = l[1]
    v = l[8:]
    y = l[3]
    fv_chelsea.append((k, v, y))

fv_chelsea[0]


('Andreas Christensen',
 [0.7166033051475624,
  0.7177921019057114,
  0.750178000200992,
  0.7518605928826311,
  -0.6284423380188117,
  -0.6091892517396329,
  -0.6305256077963044,
  -0.20680725389634816,
  -0.26178344921910174,
  -0.769776721433745,
  -0.23827472011094628,
  -0.6497387269975817,
  -0.5817543886600961,
  -0.7732420479348917,
  -0.6271528371827656,
  -0.7603489778714981,
  -0.36033145086564616,
  -0.3297020599587683,
  -0.6015016433751885,
  -0.4905482025518927,
  -0.5512377646282854,
  -0.6029613083883805,
  -0.6859381107771853,
  -0.5260907422292711,
  -0.6718588276133368,
  'Matches',
  -0.29611804549814663,
  -0.634289638534483,
  -1.1439558485172903,
  -0.6198344257446633,
  -0.801877429524663,
  -0.7200194048572042,
  -1.212228308339555,
  -1.571520515635263,
  -0.35620687880494933,
  -0.3732725612782309,
  -0.7513613660332892,
  -0.7567283787497954,
  0.8586817666224752,
  0.6820627862911928,
  1.0812180116415202,
  1.0705422796553574,
  0.8198873251746304,
  0.03